In [3]:
import pandas as pd
import numpy as np
import re
from datetime import date
import readchar
import re
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO

def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0    
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)


    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text

Boleto_TTC = convert_pdf_to_txt('BOL 21932.pdf')
Boleto_TTC = re.sub('\n',' ',Boleto_TTC)

NF_TTC = convert_pdf_to_txt('NFs 21932 HINDALCO.pdf')
NF_TTC = re.sub('\n',' ',NF_TTC)

In [6]:
NF_TTC

'PREFEITURA DA CIDADE DO RIO DE JANEIRO  SECRETARIA MUNICIPAL  DE FAZENDA  NOTA FISCAL DE SERVIÇOS ELETRÔNICA - NFS-e  - NOTA CARIOCA -  PRESTADOR DE SERVIÇOS  Número da nota 21932     Data e Hora da Emissão: 08/03/2021 11:25:03 Código de verificação  3LND-JEPF  Insc. Estadual :  85730533  Insc. Municipal: 1978560  01089179000168  CNPJ: Nome / Razão Social: Nome Fantasia :  Endereço :  AVN BRASIL 3666--CAJU- CEP:20930040 Municipio :   TTC LOGISTICA LTDA  RIO DE JANEIRO  TTC LOGISTICA   Fone/Fax: 32951501  UF :  RJ  E-mail: nfs-eletronica@ttclogistica.com.br  17.720.994/0001-13  TOMADOR DE SERVIÇOS  Insc. Municipal:  Insc. Estadual :  HINDALCO DO BRASIL INDUSTRIA E COMERCIO DE ALUMINA  AV AV. AMERICO RENE GIANETTI S/N-SARAMENHA-  CEP:35400000 OURO PRETO       - MG  E-mail: lucas.magalhaes@adityabirla.com  UF :   Fone/Fax:  CPF/CNPJ: Razão Social: Endereço: Municipio :   OVACAO/ESTUFAGEM CNTR 07X20 ( 241ISZ1703034PROJETO 21129) - R$ 7.736,82  DISCRIMINAÇÃO DOS SERVIÇOS  Suspensao de PIS 

In [12]:
#Numero da Nota
Num_Nota = re.findall(r'[\s+]?[0-9]{5}\s+', NF_TTC)
Num_Nota[0]
#Num_Nota = re.sub('\s','',Num_Nota)


# #Data de Emissão
# Emissao_Nota = re.findall(r'[0-9]{2}/[0-9]{2}/[0-9]{4}', NF_TTC)
# Emissao_Nota = Emissao_Nota[0]

# #Valor da NF
# Valor_Nota = re.findall(r'(?=VALOR DA NOTA\s+).+?(?=Legenda)', NF_TTC)
# Valor_Nota = Valor_Nota[0]
# Valor_Nota = re.findall(r'(?=[0-9]).+?(?=\s)', Valor_Nota)
# Valor_Nota = Valor_Nota[0]

# #Nº Containers
# Num_Cntr = re.findall(r'\s[0-9]{2}[X]?[x]?[0-9]{2}\s', NF_TTC)
# Num_Cntr = Num_Cntr[0]
# Num_Cntr = re.sub('\s','',Num_Cntr)
# Num_Cntr = re.findall(r'(?=[0-9])[0-9]', Num_Cntr)
# Num_Cntr = Num_Cntr[1]

# #Invoice
# Invoice_Nota = re.findall(r'(?=CNTR.+).+?(?=\))', NF_TTC)
# Invoice_Nota = Invoice_Nota[0]
# Invoice_Nota = re.findall(r'\s[0-9]{5}', Invoice_Nota)
# Invoice_Nota = Invoice_Nota[0]
# Invoice_Nota = re.findall(r'[0-9]{5}', Invoice_Nota)
# Invoice_Nota = Invoice_Nota[0]

' 21932 '

In [11]:
#Valor do Boleto
Valor_Bol = re.findall(r'(?=Valor do Documento\s+).+?(?=\s+\(-\))', Boleto_TTC)
Valor_Bol = Valor_Bol[0]
Valor_Bol = re.findall(r'(?=[0-9]).+', Valor_Bol)
Valor_Bol = Valor_Bol[0]

#Vencimento do boleto
Venc_Bol = re.findall(r'(?=Vencimento\s+).+?(?=\s+[A-Z])', Boleto_TTC)
Venc_Bol = Venc_Bol[0]
Venc_Bol = re.findall(r'(?=[0-9]).+', Venc_Bol)
Venc_Bol = Venc_Bol[0]

#Numero do boleto
Num_Bol = re.findall(r'\s[0-9]{5}\s.+?(?=\s)', Boleto_TTC)
Num_Bol = Num_Bol[0]
Num_Bol = re.sub('\s','',Num_Bol)
Num_Bol = re.sub('Uso','',Num_Bol)

In [12]:
TTC = {'INVOICE': Invoice_Nota, 'NF': Num_Bol, 'Valor da NF': Valor_Bol, 'Emissão': Emissao_Nota, 'VENCIMENTO': Venc_Bol, 'QUANTIDADE CONTAINERS': Num_Cntr,'del': [3]}
TTC = pd.DataFrame(data=TTC)
TTC = TTC.drop(TTC.columns[[6]], axis=1, inplace=False)

In [13]:
TTC_automatico = pd.read_excel('TTC_Tabela_automatica.xlsx')
TTC_automatico = pd.concat([TTC_automatico, TTC[['INVOICE','NF','Valor da NF','Emissão','VENCIMENTO','QUANTIDADE CONTAINERS']]],ignore_index=True)
#TTC_automatico = TTC_automatico.drop(TTC.columns[[0]], axis=1, inplace=False)

TTC_automatico.to_excel('TTC_Tabela_automatica.xlsx',index = False)

Compile aqui o Programa Completo!

In [19]:
#Numero da Nota
Num_Nota = re.findall(r'\s[0-9]{5}[\s+]?', NF_TTC)
Num_Nota = Num_Nota[0]
Num_Nota = re.sub('\s','',Num_Nota)

#Data de Emissão
Emissao_Nota = re.findall(r'[0-9]{2}/[0-9]{2}/[0-9]{4}', NF_TTC)
Emissao_Nota = Emissao_Nota[0]

#Valor da NF
Valor_Nota = re.findall(r'(?=VALOR DA NOTA\s+).+?(?=Legenda)', NF_TTC)
Valor_Nota = Valor_Nota[0]
Valor_Nota = re.findall(r'(?=[0-9]).+?(?=\s)', Valor_Nota)
Valor_Nota = Valor_Nota[0]

#Nº Containers
Num_Cntr = re.findall(r'\s[0-9]{2}[X]?[x]?[0-9]{2}\s', NF_TTC)
Num_Cntr = Num_Cntr[0]
Num_Cntr = re.sub('\s','',Num_Cntr)
Num_Cntr = re.findall(r'(?=[0-9])[0-9]', Num_Cntr)
Num_Cntr = Num_Cntr[1]

#Invoice
Invoice_Nota = re.findall(r'(?=CNTR.+).+?(?=\))', NF_TTC)
Invoice_Nota = Invoice_Nota[0]
Invoice_Nota = re.findall(r'\s[0-9]{5}', Invoice_Nota)
Invoice_Nota = Invoice_Nota[0]
Invoice_Nota = re.findall(r'[0-9]{5}', Invoice_Nota)
Invoice_Nota = Invoice_Nota[0]

#Valor do Boleto
Valor_Bol = re.findall(r'(?=Valor do Documento\s+).+?(?=\s+\(-\))', Boleto_TTC)
Valor_Bol = Valor_Bol[0]
Valor_Bol = re.findall(r'(?=[0-9]).+', Valor_Bol)
Valor_Bol = Valor_Bol[0]

#Vencimento do boleto
Venc_Bol = re.findall(r'(?=Vencimento\s+).+?(?=\s+[A-Z])', Boleto_TTC)
Venc_Bol = Venc_Bol[0]
Venc_Bol = re.findall(r'(?=[0-9]).+', Venc_Bol)
Venc_Bol = Venc_Bol[0]

#Numero do boleto
Num_Bol = re.findall(r'\s[0-9]{5}\s.+?(?=\s)', Boleto_TTC)
Num_Bol = Num_Bol[0]
Num_Bol = re.sub('\s','',Num_Bol)
Num_Bol = re.sub('Uso','',Num_Bol)

TTC = {'INVOICE': Invoice_Nota, 'NF': Num_Bol, 'Valor da NF': Valor_Bol, 'Emissão': Emissao_Nota, 'VENCIMENTO': Venc_Bol, 'QUANTIDADE CONTAINERS': Num_Cntr,'del': [3]}
TTC = pd.DataFrame(data=TTC)
TTC = TTC.drop(TTC.columns[[6]], axis=1, inplace=False)

TTC_automatico = pd.read_excel('TTC_Tabela_automatica.xlsx')
TTC_automatico = pd.concat([TTC_automatico, TTC[['INVOICE','NF','Valor da NF','Emissão','VENCIMENTO','QUANTIDADE CONTAINERS']]],ignore_index=True)
#TTC_automatico = TTC_automatico.drop(TTC.columns[[0]], axis=1, inplace=False)

TTC_automatico.to_excel('TTC_Tabela_automatica.xlsx',index = False)